In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium
import os

In [ ]:

carpeta_co = "Datasets\Datos_SIATA_Aire_co.json"

with open(carpeta_co, "r", encoding="utf-8") as f:data = json.load(f)

registros = []

        
for estacion in data:
    lat = estacion.get("latitud")
    lon = estacion.get("longitud")
    codigo = estacion.get("codigoSerial")
    nombre = estacion.get("nombre")
    nombreCorto = estacion.get("nombreCorto")
    datos = estacion.get("datos", [])

    for entrada in datos:
        registros.append({
            "codigoSerial": codigo,
            "nombre": nombre,
            "nombreCorto": nombreCorto,
            "latitud": lat,
            "longitud": lon,
            "contaminante": entrada.get("variableConsulta"),
            "fecha": entrada.get("fecha"),
            "calibracion": entrada.get("valor"), # Ojooo esto es calibración 
            "valor": entrada.get("calidad") # Y esto calidad, osea que la calidad se llama valor y la calibración valga la redundancia calibración 
        })

# Convertimos a DataFrame como antes
df = pd.DataFrame(registros)
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
df["calibracion"] = pd.to_numeric(df["calibracion"], errors='coerce')


df.loc[df["calibracion"] == -9999, 'valor'] = np.nan

print(df.head(20))


In [ ]:
validos_por_estacion = df.groupby('nombreCorto')['valor'].apply(lambda x: x.notna().mean() * 100)
validos_por_contaminante = df.groupby('contaminante')['valor'].apply(lambda x: x.notna().mean() * 100)
print(f"Porcentaje datos válidos por estación\n{validos_por_estacion}\n\nPorcentaje datos válidos por contaminante{validos_por_contaminante}")

In [ ]:
# Obtener contaminante único (ya que el archivo es solo de uno)
contaminante_unico = df['contaminante'].unique()[0] if not df['contaminante'].isna().all() else "Desconocido"



In [ ]:
df['mes'] = df['fecha'].dt.to_period("M")
promedios_mensuales = df.groupby('mes')['valor'].mean().reset_index()
promedios_mensuales['mes'] = promedios_mensuales['mes'].astype(str)

# Graficar
plt.figure(figsize=(12,6))
sns.lineplot(data=promedios_mensuales, x='mes', y='valor', marker='o')
plt.title(f'Promedios mensuales todas las estaciones - {contaminante_unico.upper()}')
plt.xlabel('Mes')
plt.ylabel('Concentración')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# 5. Mapa estático de estaciones segun datos validos



# Media por estacion
media_estaciones = df.groupby(['codigoSerial', 'nombre', 'latitud', 'longitud'])['valor'].mean().reset_index()

validos_por_estacion = df.groupby('codigoSerial')['valor'].apply(lambda x: x.notna().mean() * 100)

media_estaciones['porcentaje_validos'] = media_estaciones['codigoSerial'].map(validos_por_estacion)


mapa = folium.Map(location=[6.25, -75.6], zoom_start=11)

for _, row in media_estaciones.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=6,
        popup=(
            f"Estación {row['nombre']}\n"
            f"Media: {row['valor']:.2f}\n"
            f"Válidos: {row['porcentaje_validos']:.1f}%"
        ),
        color = 'green' if row['porcentaje_validos'] >= 51 else 'red',
        fill=True,
    ).add_to(mapa)

mapa

In [ ]:
# mapa por estacion según concentracion de la partícula

media_estaciones = df.groupby(['codigoSerial', 'nombre', 'latitud', 'longitud'])['valor'].mean().reset_index()

# Porcentaje de datos válidos por estación
validos_por_estacion = df.groupby('codigoSerial')['valor'].apply(lambda x: x.notna().mean() * 100)
media_estaciones['porcentaje_validos'] = media_estaciones['codigoSerial'].map(validos_por_estacion)

# Clasificación de color según niveles de concentración promedio
def clasificar_calidad_aire(valor, contaminante):
    if pd.isna(valor):
        return "Sin datos"
    
    contaminante = contaminante.lower()
    
    if contaminante == 'pm25':
        if valor <= 12:
            return "Bueno"
        elif valor <= 35.4:
            return "Aceptable"
        elif valor <= 55.4:
            return "No saludable sensibles"
        elif valor <= 150.4:
            return "No saludable"
        elif valor <= 250.4:
            return "Muy no saludable"
        else:
            return "Peligroso"

    elif contaminante == 'pm10':
        if valor <= 54:
            return "Bueno"
        elif valor <= 154:
            return "Aceptable"
        elif valor <= 254:
            return "No saludable sensibles"
        elif valor <= 354:
            return "No saludable"
        elif valor <= 424:
            return "Muy no saludable"
        else:
            return "Peligroso"

    elif contaminante == 'co':
        if valor <= 4.4:
            return "Bueno"
        elif valor <= 9.4:
            return "Aceptable"
        elif valor <= 12.4:
            return "No saludable sensibles"
        elif valor <= 15.4:
            return "No saludable"
        elif valor <= 30.4:
            return "Muy no saludable"
        else:
            return "Peligroso"

    elif contaminante == 'o3':
        if valor <= 0.054:
            return "Bueno"
        elif valor <= 0.070:
            return "Aceptable"
        elif valor <= 0.085:
            return "No saludable sensibles"
        elif valor <= 0.105:
            return "No saludable"
        elif valor <= 0.200:
            return "Muy no saludable"
        else:
            return "Peligroso"

    elif contaminante == 'no2':
        if valor <= 0.053:
            return "Bueno"
        elif valor <= 0.100:
            return "Aceptable"
        elif valor <= 0.360:
            return "No saludable sensibles"
        elif valor <= 0.649:
            return "No saludable"
        elif valor <= 1.249:
            return "Muy no saludable"
        else:
            return "Peligroso"

    elif contaminante == 'so2':
        if valor <= 0.035:
            return "Bueno"
        elif valor <= 0.075:
            return "Aceptable"
        elif valor <= 0.185:
            return "No saludable sensibles"
        elif valor <= 0.304:
            return "No saludable"
        elif valor <= 0.604:
            return "Muy no saludable"
        else:
            return "Peligroso"

    elif contaminante == 'no' or contaminante == 'pm1':
        return "Sin estándar oficial"

    else:
        return "Contaminante desconocido"
    
def color_por_categoria(categoria):
    colores = {
        "Bueno": "green",
        "Aceptable": "yellow",
        "No saludable sensibles": "orange",
        "No saludable": "red",
        "Muy no saludable": "purple",
        "Peligroso": "maroon",
        "Sin datos": "gray",
        "Sin estándar oficial": "blue",
        "Contaminante desconocido": "black"
    }
    return colores.get(categoria, "black")


# Crear mapa
mapa = folium.Map(location=[6.25, -75.6], zoom_start=11)

for _, row in media_estaciones.iterrows():
    categoria = clasificar_calidad_aire(row['valor'], contaminante_unico) 
    color = color_por_categoria(categoria)
    radio = 4 + (row['valor'] / 5) if not pd.isna(row['valor']) else 4
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=radio,
        popup=folium.Popup(
            f"<b>{row['nombre']}</b><br>"
            f"Media: {row['valor']:.2f}<br>"
            f"Categoría: {categoria}<br>"
            f"Válidos: {row['porcentaje_validos']:.1f}%",
            max_width=300
        ),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7
    ).add_to(mapa)

# Mostrar mapa en notebook
mapa

In [ ]:
df['dia_semana'] = df['fecha'].dt.weekday  # Lunes=0, Domingo=6
df['es_fin_semana'] = df['dia_semana'] >= 5

In [ ]:
eventos_especiales = {
    '2021-12-24': 'Navidad',
    '2021-12-31': 'Año Nuevo',
    '2022-01-06': 'Día de Reyes',
    '2022-04-10': 'Semana Santa',
    '2022-04-15': 'Semana Santa',
    '2022-07-20': 'Independencia',
    '2022-08-07': 'Feria de las Flores',
    '2022-08-08': 'Feria de las Flores',
    '2022-09-22': 'Día sin carro',
    '2022-10-31': 'Halloween',
    '2022-12-07': 'Día de las Velitas',
    '2022-12-24': 'Navidad',
    '2022-12-31': 'Año Nuevo',
    # Puedes seguir agregando...
}


In [ ]:
df['fecha_str'] = df['fecha'].dt.strftime('%Y-%m-%d')
df['evento'] = df['fecha_str'].map(eventos_especiales).fillna("Normal")
df

Datos muy importantes 

In [ ]:
#Semana vs fin de semana

promedios_dia = df.groupby('es_fin_semana')['valor'].mean()
print(promedios_dia)


In [ ]:
#fechas especiales vs fechas normales

promedios_evento = df.groupby('evento')['valor'].mean()
print(promedios_evento)


In [ ]:
df['dia_semana'] = df['fecha'].dt.weekday
df['tipo_dia'] = df['dia_semana'].apply(lambda x: 'Fin de semana' if x >= 5 else 'Entre semana')

sns.barplot(data=df, x='tipo_dia', y='valor')
plt.title(f'{contaminante_unico.upper()} - Promedio por tipo de día')
plt.ylabel('Concentración')
plt.show()
#Semana vs fin de semana

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='evento', y='valor')
plt.title(f'{contaminante_unico.upper()} - Concentración por evento')
plt.ylabel('Concentración (μg/m³ o ppm)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
#fechas especiales vs fechas normales

In [ ]:
resumen = df.groupby(['tipo_dia', 'evento'])['valor'].mean().unstack()
resumen


In [ ]:
#estaciones con peor aire

media_top = media_estaciones[['nombre', 'valor']].sort_values(by='valor', ascending=False)
print(media_top.head(10))


In [ ]:
plt.figure(figsize=(12, 6))
estaciones_ordenadas = media_estaciones.sort_values(by='valor', ascending=False)
sns.barplot(data=estaciones_ordenadas, x='valor', y='nombre', palette='Spectral')
plt.title(f'{contaminante_unico.upper()} - Concentración promedio por estación')
plt.xlabel('Concentración promedio')
plt.ylabel('Estación')
plt.tight_layout()
plt.show()


In [ ]:
media_plot = media_estaciones.dropna(subset=['valor'])


import plotly.express as px

fig = px.scatter_mapbox(
    media_plot,
    lat="latitud", lon="longitud",
    color="valor",
    size="valor",
    hover_name="nombre",
    color_continuous_scale="YlOrRd",
    size_max=15,
    zoom=10,
    mapbox_style="carto-positron"
)
fig.update_layout(title=f'{contaminante_unico.upper()} - Concentración por estación')
fig.show()


In [ ]:
limites_normativos = {
    'pm25': 25,   # μg/m³ (24h) - OMS
    'pm10': 50,   # μg/m³ (24h) - OMS
    'ozono': 0.051,  # ppm (8h) - OMS 100 μg/m³ → 0.051 ppm aprox
    'co': 4.0,    # ppm (24h) - OMS
    'no2': 0.025, # ppm (1h) - OMS
    'so2': 0.005, # ppm (24h) - OMS
    # otros...
}


In [ ]:
contaminante = contaminante_unico.lower()
limite = limites_normativos.get(contaminante, None)

if limite:
    df['excede_limite'] = df['valor'] > limite
    excedencias = df['excede_limite'].mean() * 100
    print(f"📌 Porcentaje de registros que exceden el límite OMS para {contaminante.upper()}: {excedencias:.2f}%")
else:
    print("No hay límite definido para este contaminante.")


In [ ]:
#gafico de dias que exceden

excesos_diarios = df.groupby(df['fecha'].dt.date)['excede_limite'].mean()

plt.figure(figsize=(12, 4))
excesos_diarios.plot(kind='bar', color='red')
plt.title(f'Días con excedencias de {contaminante.upper()} según OMS')
plt.ylabel('% de registros diarios que exceden')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
#comparar promedios mensuales con los limites
plt.figure(figsize=(12,6))
sns.lineplot(data=promedios_mensuales, x='mes', y='valor', marker='o')
if limite:
    plt.axhline(y=limite, color='red', linestyle='--', label='Límite OMS')
plt.title(f'{contaminante_unico.upper()} - Promedio mensual')
plt.xlabel('Mes')
plt.ylabel('Concentración')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
#variacion por hora del dia
df['hora'] = df['fecha'].dt.hour

horas = df.groupby('hora')['valor'].mean().reset_index()

plt.figure(figsize=(10,5))
sns.lineplot(data=horas, x='hora', y='valor', marker='o')
plt.title(f'{contaminante_unico.upper()} - Promedio por hora del día')
plt.xlabel('Hora')
plt.ylabel('Concentración')
plt.xticks(range(0,24))
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
#Top 10 dias mas contaminados
top_dias = df.groupby(df['fecha'].dt.date)['valor'].mean().sort_values(ascending=False).head(10)
print("Días con mayor concentración promedio:")
print(top_dias)


In [ ]:
promedio_eventos = df.groupby('evento')['valor'].mean().sort_values(ascending=False)

# Convertir a DataFrame y ordenar por valor (descendente)
evento_df = promedio_eventos.reset_index()
evento_df.columns = ['evento', 'valor']
evento_df = evento_df.sort_values(by='valor', ascending=False).reset_index(drop=True)

# Crear una paleta personalizada basada en el número de eventos
paleta = sns.color_palette("Reds", n_colors=len(evento_df))

# Graficar
plt.figure(figsize=(10, 5))
ax = sns.barplot(
    data=evento_df,
    x='evento', y='valor',
    palette=paleta[::-1]  # ahora sí, inverso en orden decreciente
)

# Agregar etiquetas
for i, val in enumerate(evento_df['valor']):
    ax.text(i, val + 0.05, f'{val:.2f}', ha='center', va='bottom')

plt.title(f'{contaminante_unico.upper()} - Promedio de concentración por evento')
plt.ylabel('Concentración')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
stats_eventos = df.groupby('evento')['valor'].agg(['count', 'mean', 'std', 'min', 'max']).sort_values(by='mean', ascending=False)
print(stats_eventos)


In [ ]:
# Promedio por estación
top_estaciones = df.groupby('nombre')['valor'].mean().sort_values(ascending=False).head(10).reset_index()

# Paleta de colores (más alta concentración = más oscuro)
paleta = sns.color_palette("Reds", n_colors=len(top_estaciones))

# Gráfico
plt.figure(figsize=(10, 5))
ax = sns.barplot(
    data=top_estaciones,
    x='nombre', y='valor',
    palette=paleta[::-1]
)

# Etiquetas
for i, val in enumerate(top_estaciones['valor']):
    ax.text(i, val + 0.05, f'{val:.2f}', ha='center', va='bottom')

plt.title(f'{contaminante_unico.upper()} - Top 10 estaciones con mayor concentración')
plt.ylabel('Concentración')
plt.xlabel('Estación')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Crear columna de mes
df['mes'] = df['fecha'].dt.to_period('M').astype(str)

# Agrupamos por estación y mes
pivot_df = df.groupby(['nombre', 'mes'])['valor'].mean().reset_index()

# Reorganizar en forma de matriz
pivot_table = pivot_df.pivot(index='nombre', columns='mes', values='valor')

# Crear heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(pivot_table, cmap="Reds", linewidths=0.5, linecolor='gray', annot=False, fmt=".1f", cbar_kws={'label': 'Concentración'})
plt.title(f'{contaminante_unico.upper()} - Concentración mensual por estación')
plt.xlabel('Mes')
plt.ylabel('Estación')
plt.tight_layout()
plt.show()


In [ ]:
import plotly.express as px

# Asegurar que el mes esté como string
df['mes'] = df['fecha'].dt.to_period("M").astype(str)

# Calcular promedio mensual por estación
mapa_mensual = df.groupby(['nombre', 'latitud', 'longitud', 'mes'])['valor'].mean().reset_index()

media_mensual_limpio = mapa_mensual.dropna(subset=['valor'])

# Crear mapa animado
fig = px.scatter_mapbox(
    media_mensual_limpio,
    lat="latitud", lon="longitud",
    color="valor",
    size="valor",
    animation_frame="mes",
    hover_name="nombre",
    color_continuous_scale="Reds",
    size_max=15,
    zoom=10,
    mapbox_style="carto-positron",
    title=f'{contaminante_unico.upper()} - Concentración mensual por estación'
)

fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
